![i2b2 Logo](images/transmart-logo.png)

# Using LLM + Embeddings to Search Patient Notes (RAG - Basics)

This notebook demonstrates how to use **local embeddings** and a **Retrieval-Augmented Generation (RAG)** approach to search and analyze clinical notes stored in an i2b2-like format. You'll learn how to decode raw notes, embed them using the MiniLM model, perform semantic search with FAISS, and generate structured clinical responses using a local LLM (e.g., Qwen or LLaMA 3 via Ollama).

### 🔍 Key Concepts Covered

- Decoding BinHex-encoded clinical notes
- Creating semantic vector embeddings with `MiniLM`
- Storing embeddings in a FAISS vector store (in memory)
- Performing similarity search and interpreting cosine similarity scores
- Filtering to include only the **most relevant and recent** patient notes
- Injecting retrieved context into a structured prompt template
- Using a local LLM (Ollama) to generate clinically relevant summaries

Each cell builds on the previous one to demonstrate a complete, hands-on RAG pipeline adapted for **clinical informatics** use cases using familiar i2b2-style data.


## 1. Prepare Data for Embedding

Before we can search and analyze clinical notes using vector similarity, we need to prepare the data:

- **1.1**: Load the i2b2-mimicking dataset containing BinHex-encoded clinical notes.
- **1.2**: Decode the notes and add a new column (`note_text`) with plain-text content.

This prepares the dataset for the next step, where we will embed the notes into a vector space using a local MiniLM model.


In [59]:
# -----------------------------------------------------------
# 1.1. Load and Explore Visit Data from i2b2-Mimicking CSV
# -----------------------------------------------------------
# This cell loads clinical visit data from a CSV that simulates the i2b2
# `visit_dimension` table. Each row contains metadata and a clinical note
# encoded in BinHex format.

# Fields included:
# - encounter_num: Unique visit ID
# - patient_num: Patient identifier
# - start_date, end_date: Visit dates
# - location_cd, location_path: Care location details
# - visit_blob: BinHex-encoded clinical note text

import pandas as pd
from mistune import markdown

# Define the path to the simulated i2b2 CSV file
csv_path = "datafiles/i2b2_encounter_table.csv"

# Load the data into a pandas DataFrame
df = pd.read_csv(csv_path)

# Display the first 10 rows to inspect the structure
df.head(10)


,encounter_num,patient_num,start_date,end_date,inout_cd,location_cd,location_path,visit_blob
0,475303,1000000001,01/16/2003,01/16/2003,O,ASTHMA_CLINIC,\Hospital\Clinic\Pulmonary\Asthma\\,0x2A2A566973697420496E666F726D6174696F6E3A2A2A...
1,479681,1000000001,03/29/2007,03/29/2007,O,ASTHMA_CLINIC,\Hospital\Clinic\Pulmonary\Asthma\\,0x2A2A566973697420496E666F726D6174696F6E3A2A2A...
2,480315,1000000001,09/20/2007,09/20/2007,O,ASTHMA_CLINIC,\Hospital\Clinic\Pulmonary\Asthma\\,0x2A2A566973697420496E666F726D6174696F6E3A2A2A...
3,480903,1000000001,03/04/2008,03/04/2008,O,ASTHMA_CLINIC,\Hospital\Clinic\Pulmonary\Asthma\\,0x5468697320697320612032332D796561722D6F6C6420...
4,481398,1000000001,08/11/2008,08/11/2008,O,ASTHMA_CLINIC,\Hospital\Clinic\Pulmonary\Asthma\\,0x566973697420496E666F726D6174696F6E3A20202020...
5,482655,1000000001,05/18/2009,05/18/2009,O,ASTHMA_CLINIC,\Hospital\Clinic\Pulmonary\Asthma\\,0x566973697420496E666F726D6174696F6E3A0A202020...
6,471658,1000000002,04/17/1998,04/17/1998,O,GEN_MED_OUTPATIENT,\Hospital\Outpatient\GeneralMedicine\\,0x2A2A566973697420496E666F726D6174696F6E3A2A2A...
7,472076,1000000002,01/04/1999,01/04/1999,O,GEN_MED_OUTPATIENT,\Hospital\Outpatient\GeneralMedicine\\,0x2A2A566973697420496E666F726D6174696F6E3A2A2A...
8,472473,1000000002,08/12/1999,08/12/1999,O,GEN_MED_OUTPATIENT,\Hospital\Outpatient\GeneralMedicine\\,0x2A2A566973697420496E666F726D6174696F6E3A2A2A...
9,472692,1000000002,12/01/1999,12/01/1999,O,GEN_MED_OUTPATIENT,\Hospital\Outpatient\GeneralMedicine\\,0x2A2A566973697420496E666F726D6174696F6E3A2A2A...


In [60]:
# -----------------------------------------------------------
# 1.2. Decode BinHex Clinical Notes and Prepare Text Corpus
# -----------------------------------------------------------
# This cell decodes the clinical notes stored in BinHex format and
# adds a new `note_text` column containing plain-text notes.
# These will be used for embedding in the next step.

import binascii
from IPython.display import display, Markdown

# Function to decode a single BinHex string
def decode_note(hex_blob):
    hex_str = hex_blob.replace("0x", "")
    return binascii.unhexlify(hex_str).decode("utf-8", errors="ignore")

# Apply decoding to all rows
df["note_text"] = df["visit_blob"].apply(decode_note)

# Display the first 10 decoded records
display(df.head(10))

# Display an example decoded note
example_index = 10
display(Markdown(f"### Decoded Note Example (Row {example_index} of {len(df)}):\n\n```text\n{df['note_text'][example_index]}\n```"))


,encounter_num,patient_num,start_date,end_date,inout_cd,location_cd,location_path,visit_blob,note_text
0,475303,1000000001,01/16/2003,01/16/2003,O,ASTHMA_CLINIC,\Hospital\Clinic\Pulmonary\Asthma\\,0x2A2A566973697420496E666F726D6174696F6E3A2A2A...,**Visit Information:**\n - Patient ID: 10000...
1,479681,1000000001,03/29/2007,03/29/2007,O,ASTHMA_CLINIC,\Hospital\Clinic\Pulmonary\Asthma\\,0x2A2A566973697420496E666F726D6174696F6E3A2A2A...,**Visit Information:**\n- Patient ID: 10000000...
2,480315,1000000001,09/20/2007,09/20/2007,O,ASTHMA_CLINIC,\Hospital\Clinic\Pulmonary\Asthma\\,0x2A2A566973697420496E666F726D6174696F6E3A2A2A...,**Visit Information:**\n\n - Patient ID: 10...
3,480903,1000000001,03/04/2008,03/04/2008,O,ASTHMA_CLINIC,\Hospital\Clinic\Pulmonary\Asthma\\,0x5468697320697320612032332D796561722D6F6C6420...,This is a 23-year-old Black female who has bee...
4,481398,1000000001,08/11/2008,08/11/2008,O,ASTHMA_CLINIC,\Hospital\Clinic\Pulmonary\Asthma\\,0x566973697420496E666F726D6174696F6E3A20202020...,Visit Information: \n - Patient ID: 1000...
5,482655,1000000001,05/18/2009,05/18/2009,O,ASTHMA_CLINIC,\Hospital\Clinic\Pulmonary\Asthma\\,0x566973697420496E666F726D6174696F6E3A0A202020...,Visit Information:\n - Patient ID: 10000000...
6,471658,1000000002,04/17/1998,04/17/1998,O,GEN_MED_OUTPATIENT,\Hospital\Outpatient\GeneralMedicine\\,0x2A2A566973697420496E666F726D6174696F6E3A2A2A...,**Visit Information:**\n- Patient ID: 10000000...
7,472076,1000000002,01/04/1999,01/04/1999,O,GEN_MED_OUTPATIENT,\Hospital\Outpatient\GeneralMedicine\\,0x2A2A566973697420496E666F726D6174696F6E3A2A2A...,**Visit Information:**\n- Patient ID: 10000000...
8,472473,1000000002,08/12/1999,08/12/1999,O,GEN_MED_OUTPATIENT,\Hospital\Outpatient\GeneralMedicine\\,0x2A2A566973697420496E666F726D6174696F6E3A2A2A...,**Visit Information:**\n- Patient ID: 10000000...
9,472692,1000000002,12/01/1999,12/01/1999,O,GEN_MED_OUTPATIENT,\Hospital\Outpatient\GeneralMedicine\\,0x2A2A566973697420496E666F726D6174696F6E3A2A2A...,**Visit Information:**\n- Patient ID: 10000000...


### Decoded Note Example (Row 10 of 1128):

```text
**Visit Information:**
- Patient ID: 1000000002
- Encounter ID: 473302
- Visit Date: August 25, 2000

**Subjective:**

This is a 34-year-old White female who has been receiving care at our clinic for approximately nine months. She communicates primarily in Spanish and presents today for a follow-up visit regarding persistent anxiety and depressive symptoms, as well as management of chronic cough and recurrent gastrointestinal discomfort. Additionally, she is seeking advice on long-term contraceptive options following dissatisfaction with her current method.

The patient reports a persistent cough that has been ongoing for the past two weeks, producing green sputum and accompanied by intermittent fever, chills, and general fatigue. She also describes ongoing gastrointestinal discomfort characterized by epigastric pain, bloating, and nausea, particularly after meals. There have been no recent dietary changes. 

The patient is currently taking Benzonatate for cough management and Prednisone; however, these medications have not completely alleviated her symptoms. She also takes Apri for contraception and Nortriptyline Hydrochloride for neuropathic pain and mood management. She reports improvement in mood and pain control but continues to experience gastrointestinal symptoms, including nausea associated with her contraceptive pill. The patient has no known drug allergies.

Her past medical history includes chronic cervicitis and endocervicitis with recurrent abnormal Pap smears, generalized anxiety disorder, major depressive disorder, and bipolar disorder. She has not undergone any major surgeries.

The patient lives with her family and works as a homemaker. She leads a healthy lifestyle, does not smoke, drinks alcohol occasionally, and does not use recreational drugs. 

**Objective:**

The patient appears moderately ill but is not in acute distress. Vital signs include a temperature of 99.6°F, heart rate of 82 bpm, blood pressure of 118/76 mmHg, and respiratory rate of 18 breaths per minute. The respiratory examination reveals decreased breath sounds and crackles at the base of the left lung, suggesting an ongoing respiratory infection. The abdominal examination shows mild epigastric tenderness without rebound tenderness or guarding; bowel sounds are present and normal. There is no hepatosplenomegaly. The gynecological examination was not performed during this visit. The neck examination reveals no visible deformities. Cardiovascular examination indicates a regular heart rate and rhythm with no abnormal sounds. Recent lab tests show an elevated white blood cell count, indicating infection, while liver function tests are within normal limits.

**Assessment:**

1. Acute bronchitis with a likely secondary bacterial infection, as evidenced by persistent cough, sputum production, and fever.
2. Chronic cervicitis and endocervicitis, necessitating further evaluation due to recurrent symptoms and abnormal Pap smear results.
3. Gastritis, likely exacerbated by oral contraceptive use and possibly dietary factors, presenting with epigastric pain, nausea, and bloating.

**Plan:**

1. The patient will continue with the current antibiotic regimen to address the acute bronchitis and will be monitored for signs of improvement or worsening infection.
2. Recommend dietary modifications and prescribe a proton pump inhibitor to manage gastritis symptoms.
3. Schedule a gynecological consultation to further evaluate abnormal Pap smear results and ongoing issues with chronic cervicitis.
4. Discuss various long-term contraceptive options, considering the side effects experienced with the current oral contraceptive, and make necessary adjustments.
5. Reinforce the importance of adhering to prescribed medications, maintaining adequate hydration, and ensuring rest. Educate the patient about recognizing signs of worsening infection and instruct to seek immediate medical attention if symptoms escalate.

**Follow-Up:**

Instruct the patient to contact the clinic if symptoms worsen or do not improve within a week, or if test results indicate further evaluation is needed. Educate the patient on the importance of adhering to the treatment plan and attending follow-up appointments to monitor progress and reassess treatment efficacy.
```

## 2. Decode, Embed and Store Clinical Notes in a FAISS Vector Store (In-Memory)

In this step, we embed full clinical notes and store them in a **FAISS** vector store, which enables efficient similarity search. We use a lightweight transformer model (`MiniLM`) to convert each note into a semantic vector.

### Steps:
- **2.1**: Embed clinical notes using the `all-MiniLM-L6-v2` model from Hugging Face.
- **2.2**: View an embedded document along with its metadata and vector representation.

### Why Use This Approach?

Storing entire notes is useful when:
- You want to preserve the full clinical context for each patient.
- Your downstream use case (e.g., summarization or structured extraction) requires complete narrative input.
- The notes are concise enough to fit within the input limits of an LLM.

This method simplifies retrieval workflows by allowing you to work with whole documents rather than fragmented chunks.

<img src="./images/rag_full.png" alt="RAG Full" width="900">




In [16]:
# -----------------------------------------------------------
# 2.1. Embed Clinical Notes Using Local MiniLM Embeddings
# -----------------------------------------------------------
# This cell encodes each clinical note into a vector using a local
# transformer model and stores those embeddings in a FAISS index for
# fast similarity search.

# Model: `sentence-transformers/all-MiniLM-L6-v2`
# - Optimized for semantic similarity tasks
# - Lightweight and fast (384-dimensional vectors)
# - Runs fully offline

# Requirements:
#   pip install langchain langchain-huggingface sentence-transformers faiss-cpu

from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

# Initialize the embedding model
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

# Prepare inputs for embedding: note text and relevant metadata
documents = df["note_text"].tolist()
metadata = df[["patient_num", "encounter_num", "start_date"]].to_dict(orient="records")

# Create a FAISS vector store from the documents
vectorstore = FAISS.from_texts(documents, embedding_model, metadatas=metadata)

print(f"✅ Successfully embedded {len(documents)} clinical notes using MiniLM.")


✅ Successfully embedded 1128 clinical notes using MiniLM.


In [32]:
# -----------------------------------------------------------
# 2.2. View a Specific Embedded Document, Metadata, and Vector
# -----------------------------------------------------------
# Select an index (e.g., id = 5) to inspect the stored document.
# This cell shows the document text, associated metadata, and
# the corresponding FAISS embedding vector.

id = 1  # You can change this index to view a different record

# Get (doc_id, Document) tuple from LangChain's docstore
doc_id, doc_example = list(vectorstore.docstore._dict.items())[id]

# Retrieve corresponding vector from FAISS
vector_example = vectorstore.index.reconstruct(id)

display(Markdown(f"### 🧾 Document ID: `{doc_id}`"))
display(Markdown(f"**Metadata:** `{doc_example.metadata}`"))

display(Markdown("**Document Text (First 500 characters):**"))
display(Markdown(f"```text\n{doc_example.page_content[:500]}...\n```"))

display(Markdown("**Embedded Vector (First 100):**"))
display(Markdown(f"```text\n{vector_example[:100]}\n```"))

### 🧾 Document ID: `3cea3cad-3d0f-4392-946a-afa3fe7618ce`

**Metadata:** `{'patient_num': 1000000001, 'encounter_num': 479681, 'start_date': '03/29/2007'}`

**Document Text (First 500 characters):**

```text
**Visit Information:**
- Patient ID: 1000000001
- Encounter ID: 479681
- Visit Date: 03/29/2007

**Subjective:**
This is a 22-year-old Black female who presents for a follow-up visit regarding her asthma management and other health concerns. She has a history of asthma, which has recently been exacerbated, as well as chronic but unspecified asthma without status asthmaticus. Today, she reports experiencing increased shortness of breath, coughing, and wheezing, particularly at night and during ph...
```

**Embedded Vector (First 100):**

```text
[-1.71678141e-02  2.57554296e-02 -3.02366652e-02  5.90960197e-02
 -5.69125786e-02 -4.99190413e-04  8.61884840e-03  8.68016854e-02
 -6.81851804e-02 -6.68402463e-02  3.30676250e-02 -3.55875865e-02
 -1.23848217e-02  7.65940845e-02 -3.75763103e-02  9.54439640e-02
  6.90067634e-02 -2.38603388e-05 -3.86461765e-02  4.25991453e-02
 -1.85658671e-02  5.61406054e-02  1.93215813e-02  2.50939541e-02
 -2.55058929e-02  2.48132050e-02  1.13040283e-02 -7.05091357e-02
 -4.29927222e-02  6.21788716e-03  3.62610035e-02  7.08533376e-02
 -2.21760888e-02  2.64465176e-02  7.54039586e-02  4.01692055e-02
 -6.26062378e-02  1.32325098e-01 -5.72330616e-02 -1.53708109e-03
 -4.10942920e-02 -4.31008749e-02 -7.04137757e-02 -1.14883482e-02
 -1.23254970e-01 -5.21464646e-02  9.94869322e-03  5.51104359e-02
  3.66243050e-02 -4.14495505e-02 -8.01567286e-02 -2.10100575e-03
 -3.75144444e-02  3.52221541e-02 -2.79712770e-02 -3.74436043e-02
 -2.92122159e-02 -2.80889533e-02 -1.86977591e-02 -5.62554412e-02
 -8.22607130e-02 -4.13189344e-02 -3.52802058e-03 -3.82440048e-03
 -3.04170419e-02  3.33530866e-02  7.54628479e-02 -6.59794062e-02
  7.56931379e-02  8.29117745e-02  1.21518439e-02  3.85913393e-03
 -3.69290859e-02  3.58938538e-02  8.32919627e-02  5.12350611e-02
  8.61314014e-02 -8.09207782e-02  8.05566385e-02 -7.17616081e-02
  4.12650108e-02  3.24600525e-02 -4.26827552e-04  6.82886988e-02
  3.33798267e-02  1.21426703e-02 -1.62483845e-02 -1.23265274e-02
 -7.30338134e-03 -1.13389842e-01  4.83014695e-02  5.49841635e-02
 -1.20847290e-02 -4.13356461e-02  1.21255375e-01  1.19486563e-02
 -5.47067188e-02  6.44720122e-02 -8.70435610e-02  2.73401849e-02]
```

## 3. Retrieving Clinical Notes with Similarity Score (RAG Retrieval)

In this section, we perform semantic search over embedded clinical notes using a FAISS vector store and a locally generated query vector. We use similarity scores to evaluate the relevance of each match to the query.

### Key Retrieval Steps:

1. **Embed a Query (Step 3.1)**
   - Converts a natural language question into a numerical vector using the same MiniLM model used to embed the notes.

2. **Similarity Search with Scores (Step 3.2)**
   - Retrieves the top-k clinical notes ranked by cosine similarity to the query.
   - Includes similarity scores for transparency and ranking.

3. **Score Threshold Filtering (Step 3.3)**
   - Filters out matches with low similarity scores.
   - Helps improve the precision and clinical relevance of the results.

### Why Use These Techniques?

Similarity search helps identify notes most relevant to a user-defined question or condition. Threshold filtering ensures:
- Only strong matches are considered for downstream tasks like summarization
- Noisy or unrelated content is excluded
- Each result can be justified based on a similarity score

<img src="./images/rag_retrieval.png" alt="RAG Retrieval" width="900">


In [35]:
# -----------------------------------------------------------
# 3.1. Embed a Query and Inspect Its Vector Representation
# -----------------------------------------------------------
# This step encodes a natural language query into a numerical vector
# using the same MiniLM model used for the clinical notes.
# This vector will be used to search for semantically similar notes.

from IPython.display import display, Markdown

# Define a sample clinical query
query = "Who has asthma and is taking Fluticasone and Albuterol?"

# Generate the embedding for the query
query_vector = embedding_model.embed_query(query)

# Display the vector and its shape
display(Markdown("### Vectorized Query"))
display(Markdown(f"`Query:` *{query}*"))
display(Markdown("**Embedding Vector (truncated):**"))
display(Markdown(f"```text\n{query_vector[:100]} ... [{len(query_vector)} dimensions]\n```"))


### Vectorized Query

`Query:` *Who has asthma and is taking Fluticasone and Albuterol?*

**Embedding Vector (truncated):**

```text
[0.03887706995010376, -0.042409393936395645, -0.05146171152591705, 0.04121880233287811, -0.025769369676709175, -0.04591357707977295, -0.0103689543902874, 0.08145193755626678, -0.057403627783060074, -0.024037929251790047, -0.03372789919376373, -0.017538271844387054, 0.011136984452605247, 0.026207493618130684, 0.05617568641901016, 0.10241179913282394, -0.010751360096037388, -0.080230213701725, -0.035150207579135895, 0.03298896923661232, -0.054257433861494064, 0.03680410236120224, -0.020248012617230415, -0.014638238586485386, -0.02407461777329445, 0.003959783352911472, -0.06827481091022491, -0.06968778371810913, -0.012535175308585167, -0.02130780555307865, 0.04345591366291046, 0.012301910668611526, 0.034768421202898026, 0.016457173973321915, -0.048210542649030685, -0.06472979485988617, -0.03769080340862274, 0.041117001324892044, -0.019274244084954262, 0.0037662305403500795, -0.021108878776431084, 0.04656617343425751, -0.008450526744127274, -0.049669861793518066, -0.04242834821343422, -0.06320502609014511, -0.01955222897231579, 0.054210860282182693, 0.1411016583442688, 0.1343386173248291, -0.020330237224698067, -0.10597763955593109, -0.017557138577103615, 0.025490937754511833, 0.01601383090019226, 0.013045622035861015, -0.060303207486867905, -0.03783290088176727, 0.00859709084033966, 0.039191484451293945, -0.08744332939386368, -0.05355598032474518, -0.05158739909529686, 0.0387701652944088, -0.016768554225564003, 0.08894018083810806, -0.04220253601670265, -0.010439642705023289, 0.04337993264198303, 0.024880973622202873, -0.04691239446401596, -0.09643535315990448, 0.001312048640102148, 0.05043662711977959, 0.06332380324602127, 0.06346283853054047, 0.00016291363863274455, -0.06721339374780655, -0.023447025567293167, -0.03179381415247917, 0.029838524758815765, 0.039168938994407654, 0.016349392011761665, 0.08783099800348282, 0.1331924945116043, -0.008966437540948391, 0.004134346731007099, 0.0647963434457779, -0.04450833424925804, -0.03666992485523224, 0.1214400976896286, -0.02252311259508133, -0.03951089456677437, 0.030040215700864792, 0.0030885825399309397, -0.04257494956254959, 0.0373033732175827, -0.009867946617305279, -0.13521036505699158, 0.04623591527342796] ... [384 dimensions]
```

In [37]:
# -----------------------------------------------------------
# 3.2. Similarity Search (Top-K Results, No Filtering)
# -----------------------------------------------------------
# This cell performs a semantic similarity search using the embedded query,
# returning the top-k most similar clinical notes along with similarity scores.

# Score interpretation:
# - 0.90 – 1.00: Highly relevant
# - 0.70 – 0.90: Strong match
# - 0.50 – 0.70: Moderate match
# - 0.30 – 0.50: Low match
# - 0.00 – 0.30: Minimal or irrelevant

from IPython.display import Markdown, display

# Define number of top results
top_k = 5

# Run similarity search
results = vectorstore.similarity_search_with_score(query, k=top_k)

# Display header
display(Markdown(f"### 🔍 Top {top_k} Most Similar Clinical Notes"))

# Iterate and display each match
for i, (doc, score) in enumerate(results):
    display(Markdown(f"---\n**Result {i+1}**  \n- **Similarity Score:** `{score:.4f}`  \n- **Patient Num:** `{doc.metadata.get('patient_num', 'N/A')}`  \n- **Encounter:** `{doc.metadata.get('encounter_num', 'N/A')}`\n\n**Note Preview:**\n```text\n{doc.page_content[:1200]}\n```"))


### 🔍 Top 5 Most Similar Clinical Notes

---
**Result 1**  
- **Similarity Score:** `0.5248`  
- **Patient Num:** `1000000005`  
- **Encounter:** `477663`

**Note Preview:**
```text
**Visit Information:**
- Patient ID: 1000000005
- Encounter ID: 477663
- Visit Date: June 21, 2005

**Subjective:**

This is a 32-year-old Hispanic female who has been receiving care at our clinic for approximately 3 months. She speaks English and presents today for a follow-up visit scheduled as part of her ongoing asthma management. The patient reports continued asthma symptoms, including persistent shortness of breath, wheezing particularly at night, and a cough that disrupts her sleep. Over the past two weeks, these symptoms have intensified despite regular usage of her asthma medications: albuterol inhaler for rescue, daily fluticasone inhaler, and nightly montelukast. She states the albuterol provides only brief relief.

She has a medical history of unspecified asthma without mention of status asthmaticus, back sprain from unspecified causes, vaginitis, and a previous high-risk pregnancy requiring special investigations. Her social history includes living in an urban environment known for high allergen exposure, working as a primary school teacher, and maintaining a non-smoking and non-drinking lifestyle. She has no known allergies.

**Objective:**

On examination, the patien
```

---
**Result 2**  
- **Similarity Score:** `0.5455`  
- **Patient Num:** `1000000011`  
- **Encounter:** `476139`

**Note Preview:**
```text
**Visit Information:**
- Patient ID: 1000000011
- Encounter ID: 476139
- Visit Date: Nov 20, 2003

**Subjective:**
This is a 54-year-old Caucasian female who speaks English and has been receiving care at our clinic. She presents today for a follow-up regarding her recurrent asthma and associated symptoms. The patient is experiencing persistent shortness of breath, wheezing, and a nocturnal cough that interrupts her sleep. These symptoms have been particularly troublesome over the last month despite adherence to her current medication regimen. She denies smoking and has reported no recent exposure to known allergens or new environmental triggers.

Her past medical history is significant for recurrent asthma, an acute myocardial infarction, hypertension, hypercholesterolemia, lumbar disc displacement leading to chronic lumbago, cervical dysplasia, and a panic disorder. Her medications include fluticasone and an albuterol inhaler for asthma, along with antihypertensive and lipid-lowering agents. She resides independently and leads an active lifestyle with a balanced diet and routine physical activity, adjusted as needed for her medical conditions.

**Objective:**
On examination, the p
```

---
**Result 3**  
- **Similarity Score:** `0.5618`  
- **Patient Num:** `1000000123`  
- **Encounter:** `475208`

**Note Preview:**
```text
**Visit Information:**
- Patient ID: 1000000123
- Encounter ID: 475208
- Visit Date: November 27, 2002

**Subjective:**
This is a 19-year-old Indian male who has been under our care for asthma management. The patient speaks German and came to the clinic on November 27, 2002, for a follow-up visit. Since his last appointment in March 2001, he has experienced an uptick in both the frequency and severity of his asthma attacks. Recently, his nocturnal asthma episodes have become more disruptive, leading to sleep disturbances and affecting his daily function and academic performance. The patient reports consistent symptoms of wheezing, shortness of breath, and chest tightness that are exacerbated by physical activities and cold weather. He has been reliant on his rescue inhaler, using it three to four times a day with only partial relief.

His medical history includes chronic asthma, which has persisted since childhood. He is currently managed with a fluticasone/salmeterol inhaler taken twice daily and albuterol on an as-needed basis. He does not use any herbal supplements. As a high school student, he lives with his parents and younger siblings. He maintains an active lifestyle but has
```

---
**Result 4**  
- **Similarity Score:** `0.5620`  
- **Patient Num:** `1000000005`  
- **Encounter:** `475726`

**Note Preview:**
```text
**Visit Information:**
- Patient ID: 1000000005
- Encounter ID: 475726
- Visit Date: July 2, 2003

**Subjective:**

This is a 30-year-old Hispanic female who is an English speaker, followed at our clinic for asthma management. Today, she presents for a routine follow-up visit. The patient reports persistent symptoms of asthma, including increased shortness of breath, wheezing, and a nocturnal cough that disrupts her sleep. She mentions that these symptoms have worsened over the past two weeks despite regular use of her asthma medications. She is diligent with her albuterol inhaler (used as needed), fluticasone inhaler (daily), and montelukast (nightly), yet finds that the albuterol offers only temporary relief.

Her medical history includes asthma with multiple past exacerbations usually triggered by allergens or respiratory infections. There are no past surgeries of note. Socially, she resides in an urban environment with high allergen exposure and works as a primary school teacher. She asserts a non-smoking and non-drinking lifestyle. No known allergies have been reported.

**Objective:**

Upon examination, the patient is in mild respiratory distress. Vital signs include a blood 
```

---
**Result 5**  
- **Similarity Score:** `0.5715`  
- **Patient Num:** `1000000011`  
- **Encounter:** `476451`

**Note Preview:**
```text
## SOAP Note

**Visit Information:**
- Patient ID: 1000000011
- Encounter ID: 476451
- Visit Date: March 16, 2004

**Subjective:**
This is a 55-year-old Caucasian female who speaks English and has been receiving care at our clinic for the past several months. She presents today for a follow-up visit primarily concerning her asthma, which has been problematic over the last month. The patient reports persistent shortness of breath, wheezing, and a nocturnal cough, hindering her sleep quality despite adherence to her prescribed medication regimen. She denies recent smoking, exposure to known allergens, or new environmental triggers.

Her past medical history includes recurrent asthma, a previous myocardial infarction, hypertension, hypercholesterolemia, lumbar disc displacement with chronic lumbago, cervical dysplasia, and a panic disorder. She is currently taking fluticasone and an albuterol inhaler for asthma management, along with antihypertensive and lipid-lowering medications. The patient lives independently and maintains an active lifestyle, adjusting her routine as necessary due to her health conditions. She follows a balanced diet and engages in regular physical activity. She 
```

In [42]:
# -----------------------------------------------------------
# 3.3. Filter Search Results by Similarity Score Threshold
# -----------------------------------------------------------
# This cell filters the top-K search results to keep only those
# with high similarity scores above a defined threshold.

# Similarity Score Threshold:
# - Only notes with scores ≥ threshold will be retained.
# - Higher scores = greater semantic similarity.

from IPython.display import Markdown, display

threshold = 0.55  # Keep notes with score ≥ 0.55

# Initialize an empty list to hold the filtered results
filtered_results = []

# Loop through each result (a tuple of Document and score)
for doc, score in results:
    # Check if the similarity score meets the threshold
    if score >= threshold:
        # If so, add it to the filtered list
        filtered_results.append((doc, score))


# Summary
display(Markdown(f"### ✅ {len(filtered_results)} of {top_k} notes passed the similarity threshold (≥ {threshold})"))

# Show filtered results
for i, (doc, score) in enumerate(filtered_results):
    display(Markdown(
        f"---\n**Filtered Match {i+1}**  \n"
        f"- **Similarity Score:** `{score:.4f}`  \n"
        f"- **Patient Num:** `{doc.metadata.get('patient_num', 'N/A')}`  \n"
        f"- **Encounter:** `{doc.metadata.get('encounter_num', 'N/A')}`\n\n"
        f"**Note Preview:**\n```text\n{doc.page_content[:1200]}\n```"
    ))


### ✅ 3 of 5 notes passed the similarity threshold (≥ 0.55)

---
**Filtered Match 1**  
- **Similarity Score:** `0.5618`  
- **Patient Num:** `1000000123`  
- **Encounter:** `475208`

**Note Preview:**
```text
**Visit Information:**
- Patient ID: 1000000123
- Encounter ID: 475208
- Visit Date: November 27, 2002

**Subjective:**
This is a 19-year-old Indian male who has been under our care for asthma management. The patient speaks German and came to the clinic on November 27, 2002, for a follow-up visit. Since his last appointment in March 2001, he has experienced an uptick in both the frequency and severity of his asthma attacks. Recently, his nocturnal asthma episodes have become more disruptive, leading to sleep disturbances and affecting his daily function and academic performance. The patient reports consistent symptoms of wheezing, shortness of breath, and chest tightness that are exacerbated by physical activities and cold weather. He has been reliant on his rescue inhaler, using it three to four times a day with only partial relief.

His medical history includes chronic asthma, which has persisted since childhood. He is currently managed with a fluticasone/salmeterol inhaler taken twice daily and albuterol on an as-needed basis. He does not use any herbal supplements. As a high school student, he lives with his parents and younger siblings. He maintains an active lifestyle but has
```

---
**Filtered Match 2**  
- **Similarity Score:** `0.5620`  
- **Patient Num:** `1000000005`  
- **Encounter:** `475726`

**Note Preview:**
```text
**Visit Information:**
- Patient ID: 1000000005
- Encounter ID: 475726
- Visit Date: July 2, 2003

**Subjective:**

This is a 30-year-old Hispanic female who is an English speaker, followed at our clinic for asthma management. Today, she presents for a routine follow-up visit. The patient reports persistent symptoms of asthma, including increased shortness of breath, wheezing, and a nocturnal cough that disrupts her sleep. She mentions that these symptoms have worsened over the past two weeks despite regular use of her asthma medications. She is diligent with her albuterol inhaler (used as needed), fluticasone inhaler (daily), and montelukast (nightly), yet finds that the albuterol offers only temporary relief.

Her medical history includes asthma with multiple past exacerbations usually triggered by allergens or respiratory infections. There are no past surgeries of note. Socially, she resides in an urban environment with high allergen exposure and works as a primary school teacher. She asserts a non-smoking and non-drinking lifestyle. No known allergies have been reported.

**Objective:**

Upon examination, the patient is in mild respiratory distress. Vital signs include a blood 
```

---
**Filtered Match 3**  
- **Similarity Score:** `0.5715`  
- **Patient Num:** `1000000011`  
- **Encounter:** `476451`

**Note Preview:**
```text
## SOAP Note

**Visit Information:**
- Patient ID: 1000000011
- Encounter ID: 476451
- Visit Date: March 16, 2004

**Subjective:**
This is a 55-year-old Caucasian female who speaks English and has been receiving care at our clinic for the past several months. She presents today for a follow-up visit primarily concerning her asthma, which has been problematic over the last month. The patient reports persistent shortness of breath, wheezing, and a nocturnal cough, hindering her sleep quality despite adherence to her prescribed medication regimen. She denies recent smoking, exposure to known allergens, or new environmental triggers.

Her past medical history includes recurrent asthma, a previous myocardial infarction, hypertension, hypercholesterolemia, lumbar disc displacement with chronic lumbago, cervical dysplasia, and a panic disorder. She is currently taking fluticasone and an albuterol inhaler for asthma management, along with antihypertensive and lipid-lowering medications. The patient lives independently and maintains an active lifestyle, adjusting her routine as necessary due to her health conditions. She follows a balanced diet and engages in regular physical activity. She 
```

## 4. Generating Structured Responses with an LLM (RAG Retrieval)

In this section, we take the clinical notes retrieved via semantic search and pass them into a Large Language Model (LLM) to generate structured, clinically meaningful responses. This is the final step in the **Retrieval-Augmented Generation (RAG)** pipeline.

### Key Steps:

1. **Creating a Prompt Template for LLM Querying (Step 4.1)**
   - Defines a reusable prompt structure for analyzing and summarizing clinical notes.
   - Ensures each response includes patient metadata and clear, structured outputs.

2. **Invoking LLM with Retrieved Context (Step 4.2)**
   - Inserts the top-retrieved clinical notes into the prompt.
   - Sends the prompt to a local LLM (e.g., Qwen2 via Ollama) for structured generation.
   - Returns a summary that directly answers the user’s medical query.

### Why This Matters

This step demonstrates how LLMs can synthesize information from real patient notes to produce:
- Patient-specific summaries
- Answered clinical questions
- Traceable outputs with structured identifiers

This capability is essential for use cases like clinical decision support, patient-facing summaries, or intelligent search interfaces.

<img src="./images/rag_generation.png" alt="RAG Generation" width="1250">


In [57]:
# -----------------------------------------------------------
# 4.1. Create a Prompt Template for LLM Querying
# -----------------------------------------------------------
# This prompt template guides the LLM to generate structured summaries
# from clinical notes retrieved via similarity search.

# It includes placeholders for:
# - {retrieved_docs}: Injects the top-matching clinical notes
# - {query}: A user-defined clinical question

# Output Expectations:
# - One structured response per patient
# - Includes metadata for traceability
# - Summarizes and answers the query based on each patient's most recent note

from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    "You are a medical assistant analyzing clinical notes. Based on the following records:\n\n"
    "{retrieved_docs}\n\n"
    "Answer the question: {query} using the following structure:\n"
    "   - Patient Num: <value>, Gender: <value>, Age: <value>, Race: <value>\n"
    "   - Visit Date: <value>\n"
    "   - Summary: One paragraph summarizing the patient note and answering the question.\n\n"
    "   - Has Asthma: <Yes/No>"
    "Instructions:\n"
    "- Show all patients that are relevant to the query.\n"
    "- Only consider the most recent note for each patient (identified by patient_num)."
)

display(prompt_template)


PromptTemplate(input_variables=['query', 'retrieved_docs'], input_types={}, partial_variables={}, template='You are a medical assistant analyzing clinical notes. Based on the following records:\n\n{retrieved_docs}\n\nAnswer the question: {query} using the following structure:\n   - Patient Num: <value>, Gender: <value>, Age: <value>, Race: <value>\n   - Visit Date: <value>\n   - Summary: One paragraph summarizing the patient note and answering the question.\n\n   - Has Asthma: <Yes/No>Instructions:\n- Show all patients that are relevant to the query.\n- Only consider the most recent note for each patient (identified by patient_num).')

In [58]:
# -----------------------------------------------------------
# 4.2. Use Retrieved Context to Invoke LLM and Generate Response
# -----------------------------------------------------------
# This cell completes the RAG workflow by injecting the top-matching clinical notes
# into a prompt template and invoking a local LLM to generate a structured response.

from langchain_ollama import ChatOllama
from IPython.display import display, Markdown

# Initialize the local LLM (ensure this model has been pulled via Ollama)
model = ChatOllama(model="qwen2")

# Prepare the context by joining top retrieved notes
retrieved_context = "\n\n---\n\n".join([doc.page_content for doc, _ in filtered_results])

# Fill in the prompt template with the retrieved notes and query
final_prompt = prompt_template.format(
    retrieved_docs=retrieved_context,
    query=query
)

# Run inference using the LLM
response = model.invoke(final_prompt)

# Display the generated response
display(Markdown("### 📋 LLM-Generated Response"))
display(Markdown(response.content))


### 📋 LLM-Generated Response

Patient Num: 1000000123, Gender: Male, Age: 19, Race: Indian
Visit Date: November 27, 2002
Summary: The patient is a 19-year-old male who has been under care for asthma management. He speaks German and experienced increased frequency and severity of asthma attacks since his last appointment in March 2001. His symptoms include wheezing, shortness of breath, and chest tightness exacerbated by physical activities and cold weather. He uses a rescue inhaler three to four times daily with only partial relief.

Patient Num: 1000000005, Gender: Female, Age: 30, Race: Hispanic
Visit Date: July 2, 2003
Summary: The patient is a 30-year-old female who presents for routine follow-up visit concerning persistent symptoms of asthma. She reports increased shortness of breath, wheezing, and nocturnal cough despite regular use of her asthma medications, including albuterol inhaler (used as needed), fluticasone inhaler (daily), and montelukast (nightly). She lives in an environment with high allergen exposure.

Patient Num: 1000000011, Gender: Female, Age: 55, Race: Caucasian
Visit Date: March 16, 2004
Summary: The patient is a 55-year-old female who has been under care for persistent asthma symptoms over the last month. She reports shortness of breath, wheezing, and nocturnal cough without improvement despite adherence to her prescribed medication regimen that includes fluticasone and albuterol inhalers.

Yes
All three patients listed above have asthma and are taking Fluticasone (an inhaled corticosteroid) and Albuterol (a short-acting bronchodilator).